In [1]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor as rfr
import matplotlib.pyplot as plt

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# Simulation for interaction selection

In [2]:
n = 100000
p = 10
X = np.random.choice([0, 1], (n, p))
y = X[:, 0] * X[:, 1] + X[:, 2] * X[:, 3]

In [3]:
rf = rfr(bootstrap=True, n_estimators=300)
rf.fit(X,  y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [4]:
from sklearn.tree import _tree
def all_tree_signed_paths(dtree, root_node_id=0):
    """
    Get all the individual tree signed paths from root node to the leaves
    for a decision tree classifier object [1]_.

    Parameters
    ----------
    dtree : DecisionTreeClassifier object
        An individual decision tree classifier object generated from a
        fitted RandomForestClassifier object in scikit learn.

    root_node_id : int, optional (default=0)
        The index of the root node of the tree. Should be set as default to
        0 and not changed by the user

    Returns
    -------
    paths : list
        Return a list containing 1d numpy arrays of the node paths
        taken from the root node to the leaf in the decsion tree
        classifier. There is an individual array for each
        leaf node in the decision tree.

    Notes
    -----
        To obtain a deterministic behaviour during fitting,
        ``random_state`` has to be fixed.

    References
    ----------
        .. [1] https://en.wikipedia.org/wiki/Decision_tree_learning

    Examples
    --------
    >>> from sklearn.datasets import load_breast_cancer
    >>> from sklearn.model_selection import train_test_split
    >>> from sklearn.ensemble import RandomForestClassifier
    >>> raw_data = load_breast_cancer()
    >>> X_train, X_test, y_train, y_test = train_test_split(
        raw_data.data, raw_data.target, train_size=0.9,
        random_state=2017)
    >>> rf = RandomForestClassifier(
        n_estimators=3, random_state=random_state_classifier)
    >>> rf.fit(X=X_train, y=y_train)
    >>> estimator0 = rf.estimators_[0]
    >>> tree_dat0 = all_tree_paths(dtree = estimator0,
                                   root_node_id = 0)
    >>> tree_dat0
    ...                             # doctest: +SKIP
    ...
    """
    #TODO: use the decision path function in sklearn to optimize the code
    # sanity CHECK
    #if type(dtree) != sklearn.tree.DecisionTreeClassifier:
    #    raise ValueError('dtree type is supposed to be sklearn.tree.tree.DecisionTreeClassifier but got %s'%type(dtree))

    # Use these lists to parse the tree structure
    children_left = dtree.tree_.children_left
    children_right = dtree.tree_.children_right

    if root_node_id is None:
        paths = []

    if root_node_id == _tree.TREE_LEAF:
        raise ValueError("Invalid node_id %s" % _tree.TREE_LEAF)

    # if left/right is None we'll get empty list anyway
    feature_id = dtree.tree_.feature[root_node_id] 
    if children_left[root_node_id] != _tree.TREE_LEAF:
        
        
        paths_left = [[str(feature_id) + '_L'] + l
                 for l in all_tree_signed_paths(dtree, children_left[root_node_id])]
        paths_right = [[str(feature_id) + '_R'] + l
                 for l in all_tree_signed_paths(dtree, children_right[root_node_id])]
        paths = paths_left + paths_right
    else:
        paths = [[]]
    return paths


In [5]:
paths = []
for tree in rf.estimators_:
    paths += all_tree_signed_paths(tree, 0)

In [6]:
def prevalance(paths, patterns):
    count = 0
    total = 0
    for path in paths:
        count += all(x in path for x in patterns) * 2 **(- len(path))
        total += 2 **(- len(path))
    return count * 1.0 / total

In [7]:
for feature_id in range(10):
    print(prevalance(paths, ['{}_L'.format(feature_id)]), prevalance(paths, ['{}_R'.format(feature_id)]))

0.42583333333333334 0.42583333333333334
0.32416666666666666 0.32416666666666666
0.3545833333333333 0.3545833333333333
0.3954166666666667 0.3954166666666667
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0


**Remark:** Although, all four features 1,2,3,4 should be totally symmetric, their appearance in the paths is not the same. This shows that those simulations do not exactly correspond to the population version. Maybe we should restrict the depth of the tree?

In [8]:
prevalance(paths, ['0_R', '1_R']),prevalance(paths, ['0_L', '1_L']),prevalance(paths, ['0_L', '1_R']),prevalance(paths, ['0_R', '1_L'])

(0.25, 0.0, 0.07416666666666667, 0.17583333333333334)

**Remark:** It is nice to see that the 25% which we get from the calculations do exactly pop up here. Also the 0% is what you would get in the population setting calculation. However, from the population perspective ('0_L', '1_R') and ('0_R','1_L') should be completely symmetric appear with probability 0.5^3 = 12.5%. We don't see this limit here. Why do we see it for the first two cases but not for the last two? 

In [9]:
prevalance(paths, ['2_R', '3_R']),prevalance(paths, ['2_L', '3_L']),prevalance(paths, ['2_L', '3_R']),prevalance(paths, ['2_R', '3_L'])

(0.25, 0.0, 0.14541666666666667, 0.10458333333333333)

In [10]:
prevalance(paths, ['0_R', '3_R']),prevalance(paths, ['0_L', '3_L']),prevalance(paths, ['0_L', '3_R']),prevalance(paths, ['0_R', '3_L'])

(0.18416666666666667,
 0.14708333333333334,
 0.14541666666666667,
 0.18583333333333332)

**Remark:** Here the theoretical population limit should be 0.5^6 + 0.5^3 ~ 14% for the first pattern.

In [11]:
prevalance(paths, ['1_R', '3_R']),prevalance(paths, ['1_L', '3_L']),prevalance(paths, ['1_L', '3_R']),prevalance(paths, ['1_R', '3_L'])

(0.139375, 0.139375, 0.14104166666666668, 0.13770833333333332)

**Remark:** These probabilities should, in the population case, be the same as above, but they are not. However, the fact that the first two probabilities are the same, seems to indicate that they correspond to some limit. Their theoretical value should be 0.5^6 + 0.5^3 = 0.140625. The observed value is similar fut not the same!

**Remark:** One source of bias in these simulations, compared to out theoretical results, might be that we are also averaging over paths within the same tree. These paths are not independent. Out results apply to an individual path, so it might make more sense to only randomly pick one path per tree and then average over trees. Or, alternatively, only pick one path at all and than average over i.i.d. replications of the data.

In [12]:
prevalance(paths, ['1_R', '2_R']),prevalance(paths, ['1_L', '2_L']),prevalance(paths, ['1_L', '2_R']),prevalance(paths, ['1_R', '2_L'])

(0.11, 0.10125, 0.10833333333333334, 0.10291666666666667)

In [13]:
prevalance(paths, ['0_R', '2_R']),prevalance(paths, ['0_L', '2_L']),prevalance(paths, ['0_L', '2_R']),prevalance(paths, ['0_R', '2_L'])

(0.12895833333333334,
 0.17479166666666668,
 0.16770833333333332,
 0.13604166666666667)